In [10]:
import numpy as np
import reboundx
import pandas as pd
import rebound
import time
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

In [11]:
def get_times(row):
    print(fcpath+row["runstring"])
    try:
        sim = rebound.Simulation.from_file(fcpath+row["runstring"])
        columns = ['t']
        features = [ sim.t ]
        print ('{0:.16f}'.format(sim.t))

    except:
        columns= ['t']
        features = [ np.nan ]
    return pd.Series(features, index=columns)

In [12]:
path = "../../csvs/Solar_feats/"
files = ["solar_efac1.45_1e9"]
for file_name in files:
    fcpath = path + "/{0}/final_conditions/fc".format(file_name)
    df= pd.read_csv(path+"/{0}/times_{1}.csv".format(file_name, file_name.split("efac")[1].split("_")[0] ))
    df = pd.concat([df, df.apply(get_times, axis = 1)], axis = 1)
    df.to_csv("../../csvs/Solar_feats/solar_{0}_1000.csv".format(file_name.split("efac")[1].split("-")[0], index_col = False ))
    print("saving {0}".format(file_name.split("_")[1]) )



../../csvs/Solar_feats//solar_efac1.45_1e9/final_conditions/fc0000000.bin
291711683.1758157610893250
../../csvs/Solar_feats//solar_efac1.45_1e9/final_conditions/fc0000000.bin
291711683.1758157610893250
../../csvs/Solar_feats//solar_efac1.45_1e9/final_conditions/fc0000001.bin
319441157.2010086178779602
../../csvs/Solar_feats//solar_efac1.45_1e9/final_conditions/fc0000002.bin
383760930.0972698330879211
../../csvs/Solar_feats//solar_efac1.45_1e9/final_conditions/fc0000003.bin
102959823.6366979479789734
../../csvs/Solar_feats//solar_efac1.45_1e9/final_conditions/fc0000004.bin
452640841.8480795025825500
../../csvs/Solar_feats//solar_efac1.45_1e9/final_conditions/fc0000005.bin
53607706.3401142507791519
../../csvs/Solar_feats//solar_efac1.45_1e9/final_conditions/fc0000006.bin
266867549.8728344738483429
../../csvs/Solar_feats//solar_efac1.45_1e9/final_conditions/fc0000007.bin
240181370.7345937490463257
../../csvs/Solar_feats//solar_efac1.45_1e9/final_conditions/fc0000008.bin
372474330.58748465

/home/naireen/anaconda2/envs/CSC401/lib/python3.6/site-packages/rebound/simulation.py:361: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


In [13]:
#load data
inst_times = pd.read_csv("../../csvs/Solar_feats/solar_1.45_1e9_1000.csv", index_col=0)
del inst_times["Unnamed: 0.1"]
inst_times.head()

,runstring,t
0,0000000.bin,2.917117e+08
1,0000001.bin,3.194412e+08
2,0000002.bin,3.837609e+08
3,0000003.bin,1.029598e+08
4,0000004.bin,4.526408e+08


In [14]:
times = inst_times["t"].astype("float64").values
print(times[:20])
redo_sims = np.where(np.isnan(times)==True)[0]
print(redo_sims)
#print(np.isnan(times[9]))

[  2.91711683e+08   3.19441157e+08   3.83760930e+08   1.02959824e+08
   4.52640842e+08   5.36077063e+07   2.66867550e+08   2.40181371e+08
   3.72474331e+08              nan   5.17638350e+07   1.58850142e+08
   1.49993373e+08   6.64547930e+08   1.55391050e+08   4.61272559e+08
   3.38553061e+08   1.50978602e+08              nan   5.45049574e+07]
[  9  18  35  57  78 101 108 116 117 121 144 155 168 172 182 190]


In [9]:
for vals in redo_sims:
    vals = 1
    #print("{0:07d}".format(vals))
    sim = rebound.SimulationArchive("../../csvs/Solar_feats/solar_efac1.45_1e9/simulation_archives/sa{0:07d}.bin".format(vals))
   
    last_snap = sim[-1]
    print(last_snap.t)
    rebx  = reboundx.Extras(last_snap)
    gr = rebx.add("gr_potential")
    gr.params["c"] = 10058.24
    last_snap.collision = "direct"
    #print(last_snap.particles[1].P*np.sqrt(21)/100.) # choose irrational number ~5%
    yr = 2*np.pi # one year is 2 pi in these units
    last_snap.simulationarchive_filename = "../../csvs/Solar_feats/solar_efac1.45_1e9/simulation_archives/sa{0:07d}.bin".format(vals)
    sim.interval=100
    #print(sim.interval)
    
    
    
    
    try:
        #sim.interval=10
        #sim.interval = 100
        start = time.time()
        last_snap.integrate(last_snap.t + 1e6+1.) # integrate for a million years   
        end = time.time()
        print("Time taken {0}".format(end - start))
        print("New_time", last_snap.t )
        #print(sim[-1].t)
        sim = rebound.SimulationArchive("../../csvs/Solar_feats/solar_efac1.45_1e9/simulation_archives/sa{0:07d}.bin".format(vals))
        print(sim[-1].t)
        #print(last_snap.t)
    except Exception as e:
        
        print(e)
        print("Time at collision", last_snap.t)
    break

319000000.0526188
Two particles collided (d < r1+r2)
Time at collision 319441157.2010086


In [ ]:
#pass in several paths, and rerun those simulations